<a href="https://colab.research.google.com/github/coder-royswarnajit/Battery_Life_Prediction/blob/main/Quantum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from qiskit_aer import AerSimulator  # Faster backend
from qiskit.utils import algorithm_globals
from qiskit.primitives import Estimator
from qiskit.circuit.library import ZZFeatureMap, EfficientSU2
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.algorithms import NeuralNetworkRegressor
from qiskit.algorithms.optimizers import ADAM  # Faster optimizer
from qiskit.algorithms.gradients import ParamShiftEstimatorGradient

# Load Data
data = pd.read_excel('device_battery_data.xlsx')

# Feature Engineering
data['alpha'] = data['Category'].map({
    'Laptop': 0.80, 'Smartwatch': 0.70, 'Tablet': 0.70,
    'Smartphone': 0.75, 'Gaming Console': 0.85
}).fillna(0.75)

data['beta'] = data['Category'].map({
    'Laptop': 0.20, 'Smartwatch': 0.30, 'Tablet': 0.30,
    'Smartphone': 0.25, 'Gaming Console': 0.15
}).fillna(0.25)

data['Battery_Degradation_Factor'] = (
    1 - (data['alpha'] * (data['Charging_Cycles'] / data['Maximum Cycles']))
    - (data['beta'] * (data['Device Age (years)'] / data['Battery Lifespan (years)']))
)

data['Average_Power_Consumption'] = ((data['Active Time per Day (hours)'] * data['Active Power Consumption (mW)']) + (data['Sleep Time per Day (hours)'] * data['Sleep Power Consumption (mW)'])) / 24

data['Battery_Capacity_mWh'] = data['Battery Capacity (mAh)'] * data['Battery Voltage (V)']

def get_temperature_factor(temp):
    if pd.isna(temp):
        return 1.0
    temp = float(temp)
    return 1.0 if 20 <= temp <= 30 else 0.85 if temp > 35 else 0.65 if temp < 0 else 1.0

data['Temperature Factor'] = data['Operating Temp (°C)'].apply(get_temperature_factor)

env_map = {'ideal': 1.0, 'low humidity/dust': 0.95, 'high humidity/dust': 0.925, 'extreme': 0.85}
data['Environmental Factor'] = data['Environmental Conditions'].str.lower().str.strip().map(env_map).fillna(1.0)

# Ensure no division by zero
data.loc[:, 'Average_Power_Consumption'] = data['Average_Power_Consumption'].replace(0, np.nan)
data.dropna(subset=['Average_Power_Consumption'], inplace=True)

# Compute Battery_Life if missing
data['Battery_Life'] = (
    data['Battery_Capacity_mWh'] * data['Battery_Degradation_Factor']
    * data['Temperature Factor'] * data['Environmental Factor']
    / data['Average_Power_Consumption']
)

data.dropna(subset=['Battery_Life'], inplace=True)

# Remove invalid values before applying MinMax scaling
data = data[data['Battery_Life'] > 0]
y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(data[['Battery_Life']].values)

# Prepare Data
features = ['Battery_Degradation_Factor', 'Average_Power_Consumption',
            'Battery_Capacity_mWh', 'Temperature Factor', 'Environmental Factor']
X = data[features].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Use PCA dynamically to retain at least 3 features (reduces QNN size)
pca = PCA(n_components=min(3, X.shape[1]))
X_reduced = pca.fit_transform(X_scaled)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42, shuffle=True)

# Quantum Neural Network
seed = 42
algorithm_globals.random_seed = seed
estimator = AerSimulator()  # Faster backend

num_qubits = max(1, X_reduced.shape[1])  # Reduce to at most 3 qubits

# Faster feature map and ansatz
feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=1, entanglement="linear")
ansatz = EfficientSU2(num_qubits, reps=1, entanglement="linear")

# Gradient computation
gradient = ParamShiftEstimatorGradient(estimator)

# Print circuit details
print("Feature Map Parameters:", len(feature_map.parameters))
print("Ansatz Parameters:", len(ansatz.parameters))

# Construct Quantum Neural Network
qnn = EstimatorQNN(
    circuit=feature_map.compose(ansatz),
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    estimator=estimator,
    gradient=gradient
)

# Faster optimizer (ADAM instead of SPSA)
optimizer = ADAM(maxiter=200, lr=0.01)

# Define neural network regressor
nn_regressor = NeuralNetworkRegressor(qnn, optimizer=optimizer)

# Batch training to speed up
batch_size = min(500, len(X_train))
X_train_batch, y_train_batch = X_train[:batch_size], y_train[:batch_size]

# Train the model
nn_regressor.fit(X_train_batch, y_train_batch)

# Cross-validation
kf = KFold(n_splits=2, shuffle=True, random_state=seed)
mse_scores, r2_scores = [], []

for train_idx, val_idx in kf.split(X_train):
    X_cv_train, X_cv_val = X_train[train_idx][:batch_size], X_train[val_idx][:batch_size]
    y_cv_train, y_cv_val = y_train[train_idx][:batch_size], y_train[val_idx][:batch_size]

    nn_regressor.fit(X_cv_train, y_cv_train)
    y_cv_pred = np.maximum(0, y_scaler.inverse_transform(nn_regressor.predict(X_cv_val).reshape(-1, 1)))

    mse_scores.append(mean_squared_error(y_cv_val, y_cv_pred))
    r2_scores.append(r2_score(y_cv_val, y_cv_pred))

# Final Model Evaluation
y_pred = np.maximum(0, y_scaler.inverse_transform(nn_regressor.predict(X_test)))  # Reverse scaling & prevent negatives
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nQuantum Neural Network Regressor Results:")
print(f"  MSE: {mse:.4f} (Avg CV MSE: {np.mean(mse_scores):.4f})")
print(f"  R²: {r2:.4f} (Avg CV R²: {np.mean(r2_scores):.4f})")

# Display sample predictions
print("\nSample Predictions:")
print("Actual:", y_scaler.inverse_transform(y_test[:10].reshape(-1, 1)))
print("Predicted:", y_pred[:10])


ImportError: cannot import name 'algorithm_globals' from 'qiskit.utils' (/usr/local/lib/python3.11/dist-packages/qiskit/utils/__init__.py)

In [2]:
!pip install qiskit qiskit-aer qiskit-machine-learning qiskit-algorithms pandas scikit-learn



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.6/231.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 3.5 MB/s eta 0:00:00


In [3]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from qiskit_aer import AerSimulator
from qiskit_machine_learning.utils import algorithm_globals
from qiskit.primitives import StatevectorEstimator
from qiskit.circuit.library import ZZFeatureMap, EfficientSU2
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.algorithms import NeuralNetworkRegressor
from qiskit_algorithms.optimizers import SPSA
from qiskit_algorithms.gradients import ParamShiftEstimatorGradient


In [4]:
from google.colab import files
uploaded = files.upload()

# Load the dataset (ensure the filename matches exactly)
data = pd.read_csv("device_battery_data.csv")  # Use the correct file name here


Saving device_battery_data.csv to device_battery_data.csv


In [5]:
# Mapping categorical variables
data['alpha'] = data['Category'].map({'Laptop': 0.80, 'Smartwatch': 0.70, 'Tablet': 0.70, 'Smartphone': 0.75, 'Gaming Console': 0.85}).fillna(0.75)
data['beta'] = data['Category'].map({'Laptop': 0.20, 'Smartwatch': 0.30, 'Tablet': 0.30, 'Smartphone': 0.25, 'Gaming Console': 0.15}).fillna(0.25)

# Compute Battery Degradation Factor
data['Battery_Degradation_Factor'] = (1 - (data['alpha'] * (data['Charging_Cycles'] / data['Maximum Cycles'])) - (data['beta'] * (data['Device Age (years)'] / data['Battery Lifespan (years)'])))

# Compute Average Power Consumption
data['Average_Power_Consumption'] = ((data['Active Time per Day (hours)'] * data['Active Power Consumption (mW)']) + (data['Sleep Time per Day (hours)'] * data['Sleep Power Consumption (mW)'])) / 24

# Compute Battery Capacity in mWh
data['Battery_Capacity_mWh'] = data['Battery Capacity (mAh)'] * data['Battery Voltage (V)']

# Function to compute Temperature Factor
def get_temperature_factor(temp):
    if pd.isna(temp):
        return 1.0
    temp = float(temp)
    return 1.0 if 20 <= temp <= 30 else 0.85 if temp > 35 else 0.65 if temp < 0 else 1.0

data['Temperature Factor'] = data['Operating Temp (°C)'].apply(get_temperature_factor)

# Environmental Factor Mapping
env_map = {'ideal': 1.0, 'low humidity/dust': 0.95, 'high humidity/dust': 0.925, 'extreme': 0.85}
data['Environmental Factor'] = data['Environmental Conditions'].str.lower().str.strip().map(env_map).fillna(1.0)

# Ensure no division by zero
data.loc[:, 'Average_Power_Consumption'] = data['Average_Power_Consumption'].replace(0, np.nan)
data.dropna(subset=['Average_Power_Consumption'], inplace=True)

# Compute Battery Life
data['Battery_Life'] = (
    data['Battery_Capacity_mWh'] * data['Battery_Degradation_Factor'] *
    data['Temperature Factor'] * data['Environmental Factor'] /
    data['Average_Power_Consumption']
)

# Remove invalid values
data = data[data['Battery_Life'] > 0]


In [7]:
y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(data[['Battery_Life']].values)

features = ['Battery_Degradation_Factor', 'Average_Power_Consumption', 'Battery_Capacity_mWh', 'Temperature Factor', 'Environmental Factor']
X = data[features].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA to retain at least 5 features
pca = PCA(n_components=min(5, X.shape[1]))
X_reduced = pca.fit_transform(X_scaled)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42, shuffle=True)


In [9]:
# Set seed for reproducibility
seed = 42
algorithm_globals.random_seed = seed

estimator = StatevectorEstimator()
num_qubits = max(1, X_reduced.shape[1])

# Define feature map and ansatz
feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=2, entanglement="full")
ansatz = EfficientSU2(num_qubits, reps=2, entanglement="full")


gradient = ParamShiftEstimatorGradient(estimator)

# Print circuit structure
print("Feature Map Parameters:", list(feature_map.parameters))
print("Ansatz Parameters:", list(ansatz.parameters))
print("Combined Circuit:")
print(feature_map.compose(ansatz).draw())

# Construct Quantum Neural Network
qnn = EstimatorQNN(
    circuit=feature_map.compose(ansatz),
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    estimator=estimator,
    gradient=gradient
)

# Define optimizer
optimizer = SPSA(maxiter=2000, learning_rate=0.02, perturbation=0.01)
nn_regressor = NeuralNetworkRegressor(qnn, optimizer=optimizer)


Feature Map Parameters: [ParameterVectorElement(x[0]), ParameterVectorElement(x[1]), ParameterVectorElement(x[2]), ParameterVectorElement(x[3]), ParameterVectorElement(x[4])]
Ansatz Parameters: [ParameterVectorElement(θ[0]), ParameterVectorElement(θ[1]), ParameterVectorElement(θ[2]), ParameterVectorElement(θ[3]), ParameterVectorElement(θ[4]), ParameterVectorElement(θ[5]), ParameterVectorElement(θ[6]), ParameterVectorElement(θ[7]), ParameterVectorElement(θ[8]), ParameterVectorElement(θ[9]), ParameterVectorElement(θ[10]), ParameterVectorElement(θ[11]), ParameterVectorElement(θ[12]), ParameterVectorElement(θ[13]), ParameterVectorElement(θ[14]), ParameterVectorElement(θ[15]), ParameterVectorElement(θ[16]), ParameterVectorElement(θ[17]), ParameterVectorElement(θ[18]), ParameterVectorElement(θ[19]), ParameterVectorElement(θ[20]), ParameterVectorElement(θ[21]), ParameterVectorElement(θ[22]), ParameterVectorElement(θ[23]), ParameterVectorElement(θ[24]), ParameterVectorElement(θ[25]), Parameter

In [ ]:
kf = KFold(n_splits=2, shuffle=True, random_state=42)
r2_scores, mse_scores = [], []

for train_idx, val_idx in kf.split(X_train):
    X_cv_train, X_cv_val = X_train[train_idx], X_train[val_idx]
    y_cv_train, y_cv_val = y_train[train_idx], y_train[val_idx]

    nn_regressor.fit(X_cv_train, y_cv_train)
    y_cv_pred = nn_regressor.predict(X_cv_val).reshape(-1, 1)
    y_cv_pred = y_scaler.inverse_transform(y_cv_pred)
    y_cv_pred = np.clip(y_cv_pred, 0, None)

    mse_scores.append(mean_squared_error(y_cv_val, y_cv_pred))
    r2_scores.append(r2_score(y_cv_val, y_cv_pred))

In [ ]:
y_pred = np.maximum(0, y_scaler.inverse_transform(nn_regressor.predict(X_test)))
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Quantum Neural Network Regressor:")
print("  MSE:", mse, "(Avg CV MSE:", np.mean(mse_scores), ")")
print("  R²:", r2, "(Avg CV R²:", np.mean(r2_scores), ")")

# Display sample predictions
print("Actual:", y_scaler.inverse_transform(y_test[:10].reshape(-1, 1)))
print("Predicted:", y_pred[:10])


# New Section

In [ ]:
!pip install qiskit-algorithms --no-cache-dir


In [ ]:
!pip install qiskit-algorithms --no-cache-dir


In [ ]:
!pip uninstall -y qiskit qiskit-aer qiskit-ibmq-provider qiskit-machine-learning qiskit-terra


In [ ]:
!python -m venv qiskit_env
!qiskit_env\Scripts\activate


In [ ]:
!pip install qiskit qiskit-aer qiskit-machine-learning --no-cache-dir


python -m venv qiskit_env
qiskit_env\Scripts\activate


In [ ]:
pip install --upgrade symengine==0.13.0

In [ ]:
pip install --upgrade qiskit-aer


In [ ]:
pip install qiskit-machine-learning


In [ ]:
pip install qiskit==0.43.2